# R coding

## SNP Pre-analysis

What I aim to do with this R script is to clean and filter all of the raw information previously given to me so that I can use it later on for more important analysis.

Firstly, here are the packages and libraries I had to use.

In [7]:
# Packages and libraries
# install.packages(c("data.table", "dplyr"))
library(data.table)
library(dplyr)



Attaching package: ‘data.table’

The following object is masked _by_ ‘.GlobalEnv’:

    .N


Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Now we enter the actual coding session.

In [ ]:
# We open the raw PHS(PopHumanScan) table
PHS.table <- read.table('Desktop/AleixCanalda/tableWOmeta.tab', header=T, sep="\t")

#We have a look at the dimensions of the table
dim(PHS.table)

# We select the regions that show us a signal for iHS or XPEHH (there should be 487 positions)
significant <- grep('iHS:|XPEHH:', PHS.table$statPop)

#Now we only use the columns we're interested in
useful.table <- PHS.table %>% select(GeneID, Type, chr, start, end, statPop)
#From all the rows we only get the ones that are significant 
useful.table <- useful.table[significant,]

Once this has been done, we want to create a new column with all of the populations represented in those regions

In [ ]:
#we create a new column where we'll write only the populations
useful.table$Pop <- NA
#What we want to do now is from the Statpop column only get the population, and not iHS or XPEHH
for (popupopu in 1:nrow(useful.table)) {
  #this next step is to make sure we're doing everything correctly
  #We separate all of the statistics with their populations (which are separated by a comma),
    #we use "as.character" to let R know that we're using characters
  splitsplit <- strsplit(as.character(useful.table[popupopu,"statPop"]),",")
  #From those stats, we only need iHS and XPEHH, and we use unlist to create a vector from the 
    #list created with split
  good <- grep('iHS:|XPEHH:', unlist(splitsplit))
  #Since we only want the populations, we erase the stats
  populations <- gsub("iHS:|XPEHH:","",unlist(splitsplit)[good])
  #We paste the populations on the new column, separated by a comma
  useful.table$Pop[popupopu]<- paste(populations, collapse=",")
}
#We save the table so that we can use it later on (that way we don't have to repeat this process constantly)
write.table(useful.table, file="/home/aleixcanalda/Desktop/AleixCanalda/goodstats.txt",
            append = FALSE, quote = F, sep = "\t",
            col.names = TRUE, row.names = F)

Now that we have our table, we have to get from the iHS results all the SNPs that are found in these regions and that are statistically significant (extremevalue==1).

In [ ]:
# Firstly we create a loop to go through every single chromosome
for (c in 1:22) {
  chromosome <- paste0("chr",c)
  # We create a subset of all the regions in PHS of that chromosome
  regions.per.chr <- useful.table %>% filter(chr==chromosome)
  
  # New loop of each region of PHS for the specific chromosome, one by one
  for(region in 1:nrow(regions.per.chr)){
    current.region <- regions.per.chr[region,]
    # Now we create a loop to go through all of the populations use a txt file with all of the populations
    for(pop in list.populations) {
      # At first we must access the file where we can finde the iHS information of the SNPs
      prova.chr.pop <- read.table(paste0('Desktop/AleixCanalda/ihs.output/', chromosome, "/", chromosome, pop,
                                         ".ihs.out.100bins.norm"), header=T, sep="\t")
      #We select those SNPs that are statistically important (extremevalue=1) and those that are found inside 
        #the regions that present LD
      posicionschr.snp <- prova.chr.pop[prova.chr.pop$extremevalue==1 & 
                                          prova.chr.pop$physicalPos>current.region$start & 
                                          prova.chr.pop$physicalPos<current.region$end, 2]
      #if we can indeed find SNPs in such regions, we will save them in a dataframe and paste it in the table 
        #that we created before
      if(length(posicionschr.snp)>0){
        region.info <- data.frame(current.region$GeneID, current.region$Type, current.region$chr,
                                  current.region$start, posicionschr.snp, pop)
        finalsnptable <- rbind(finalsnptable, region.info)
      }
    }
  }
}

#Lastly, we must name all of the columns accordingly
colnames(finalsnptable) <- c("GeneID","Type","chr","Region start","SNP Position","Population")
write.table(finalsnptable, file="/home/aleixcanalda/Desktop/AleixCanalda/finalsbptable.txt",
            append = FALSE, quote = F, sep = "\t",
            col.names = TRUE, row.names = F)

Next we need to know which is each SNP's rsID to use it later on in ensembl.
To do that, we need to download biomaRt.
The next script explains how to do so, however, snpmart works at a very slow rate and taking in account the amount of information and SNPs that I have, I decided to use an online tool named Kaviar Genomics where you obtain each rsID by simply giving it the exact location and choosing the Human Genome we're using(GRCh37 in our case) and the type of variation we're looking for (SNVs).

In [ ]:
source("https://bioconductor.org/biocLite.R")
BiocInstaller::biocLite(c("RMySQL","GenomicFeatures","VariantAnnotation","ensemblVEP"))
install.packages(c("libmariadb-client-lgpl-dev"))
BiocManager::install("biomaRt")

library(biomaRt)

# require(biomaRt)
head(listMarts())
mart  <- useMart(biomart="ensembl", 
                 dataset="hsapiens_gene_ensembl")

listAttributes(mart)[1:10,]
listFilters(snpmart)[1:10,]

snpmart <- useMart(biomart = "ENSEMBL_MART_SNP", dataset = "hsapiens_snp")
mart <- useMart(biomart = "ensembl", dataset = "hsapiens_gene_ensembl")

#But we're not using the latest version of the human genome, so we must specify that ours is GRCh37 
finalsnptable <- read.table("Desktop/AleixCanalda/finalsbptable.txt",header=T,sep="\t")

grch37 = useMart(biomart="ENSEMBL_MART_SNP", dataset = "hsapiens_snp", host="grch37.ensembl.org",
                 path="/biomart/martservice")
listDatasets(grch37)
listAttributes(grch37)
listFilters(grch37)

results <- data.frame()
 #Initialise storage vector

 for (snp in 1:nrow(finalsnptable)) {  
   chrom <- gsub(pattern = 'chr', replacement = '', x = finalsnptable[snp, 3]) 
   # Remove 'chr' for searching
    #we're looking for the allele and the refsnp_id by giving it the start and end of the SNP
    temp <- getBM(attributes = c('refsnp_id','allele'), 
                  filters = c('chr_name','start','end'), 
                  values = list(chrom,finalsnptable$`SNP Position`[snp],finalsnptable$`SNP Position`[snp]),
                  mart = grch37)
    if (nrow(temp)!=0){
      info.snp <- data.frame(chrom, finalsnptable$`SNP Position`[snp], temp)
      }
    else{
      info.snp <- c(chrom, finalsnptable$`SNP Position`[snp], NA, NA)
    }
    results <- rbind(results,info.snp)
  }

write.table(results, file="/home/aleixcanalda/Desktop/AleixCanalda/snpvepbiomart.txt",
            append = FALSE, quote = F, sep = "\t",
            col.names = TRUE, row.names = F)

In order to use the Kaviar Genomics service, we need to create a file with the position of all the available and significant SNPs in this way:

In [ ]:
#To use VEP first we need to create a new document with the information of the SNP's position which is what we'll use.
snpvep <- data.frame()

for (vep in 1:nrow(finalsnptable)) {
  chrom <- gsub(pattern = 'chr', replacement = '', x = finalsnptable$chr[vep])
  snpinfo <- data.frame(paste0(chrom,":",finalsnptable$`SNP Position`[vep]))
  snpvep <- rbind(snpvep,snpinfo)
}
write.table(snpvep, file="/home/aleixcanalda/Desktop/AleixCanalda/snpvep2.txt",
            append = FALSE, quote = F, sep = "\t",
            col.names = TRUE, row.names = F)

In [ ]:
#Now we can use the Kaviar Genomic Variant Database where we can find all of the rsIDs of the SNPs that we'll need to use them for VEP on the terminal
#But before we can use VEP we have to create a file where the only information is the rsIDs
rsidtable <- fread("/home/aleixcanalda/Desktop/AleixCanalda/vcfsnp.txt",header = T, sep = "\t")
rsids <- data.frame(rsidtable$ID)
write.table(rsids, file="/home/aleixcanalda/Desktop/AleixCanalda/rsids.txt",
            append = FALSE, quote = F, sep = "\t",
            col.names = TRUE, row.names = F)

## Variant Effect Predictor

Now that we have the rsIDs of each SNP we can use VEP to find out what consequence each SNP has and how important it may be in a genomic adaptive sense. VEP can be used online when one has few information but that is not my case, so I need to download the whole VEP package to use it on the terminal. Nevertheless, the package contains an enormous amount of information and needs a lot of internet connection, and that is not possible with my computer, but we found a solution. I have been added to the investigation group's Server so that I can use the internet and run some commands there. 

To install VEP, which is no easy task, I firstly needed to install BioConda in order to use Conda, which, by definition, is "an open source package management system and environment management system that runs on Windows, macOS and Linux. Conda quickly installs, runs and updates packages and their dependencies. Conda easily creates, saves, loads and switches between environments on your local computer. It was created for Python programs, but it can package and distribute software for any language."
Therefore, by using Conda I already had a few packages installed. 

Once the human genome information for VEP was downloaded, and by sending the rsIDs file into the remote Server, I was able to obtain all of the necessary information regarding the impact of the SNP, by using this line in bash language:

In [ ]:
#bash
./vep --cache --port 3337 -i /home/acanalda/Data/rsids.txt -o /home/acanalda/Data/vepresults.txt

Once we have all of the information on VEP, we want to determine which SNP has the most effect on the genome and which has a less effect, however can¡t reassure that, for example, an intergenic variant has a lesser effect than an intronic variant, since regulatory sequences may be involved (which is why later on I determined the Regulome score for each SNP). 

I decided to create an order of worse consequence to a more benign consequence, according to information published on Ensembl saying which consequences had more impact: high, moderate, low or modifier. I decided to differentiate each consequence inside each group using my own criteria (revised by my tutor).

In [ ]:
#Firstly we read the table with the VEP results
vepresults <- fread("/home/aleixcanalda/Desktop/AleixCanalda/VEP/vepresults.txt", header=T)
vepresults$NumConsequence <- NA
vepresults$MaxNumConsequence <- NA

vepresults[, `NumConsequence`:=as.character(`NumConsequence`)]
vepresults[, `MaxNumConsequence`:=as.character(`MaxNumConsequence`)]
#Secondly we read the text file with all of the rsIDs
ids <- readLines("/home/aleixcanalda/Desktop/AleixCanalda/rsids.txt")

#we create the table where we'll store all of the SNPs with their worst consequence
finalconseqnumbertable <- data.frame()

#we use a hierarchy to determine which effect is worser than the next one, being a stop gained the worst effect
allconseq <- c("stop_gained","stop_lost","start_lost","splice_acceptor_variant","splice_donor_variant",
               "missense_variant","splice_region_variant","incomplete_terminal_codon_variant","stop_retained_variant",
               "synonymous_variant","coding_sequence_variant","5_prime_UTR_variant","3_prime_UTR_variant",
               "mature_miRNA_variant","NMD_transcript_variant","non_coding_transcript_exon_variant",
               "non_coding_transcript_variant","intron_variant","upstream_gene_variant","downstream_gene_variant",
               "intergenic_variant")
#then we create a general table where we assign each consequence a number, 1 being the worst consequence and 21 
#the most benign
conseqnum <- data.frame(Consequence=allconseq,Impact=c(1:21))

#we create a loop where we read each rsID one by one
for (rsid in ids) {
  print(paste("-->", rsid, sep=" "))
  #from the rsID we're reading now we get all of its rows and we only look at the "Consequence" column.
    #All of the rows whose consequence is repeated is put into one single consequence using "unique"
  vector <- unlist(unique(vepresults[vepresults$`#Uploaded_variation`==rsid,7]))
  #we then look at each consequence to see if there are more than one in one line joined by a ',', so we can
    #separate them
  for (aiai in vector) {
    if (grepl(",",aiai) == TRUE) {
      new <- unlist(strsplit(aiai, ","))
      
      vector <-append(vector,new)
      vector = vector[!(vector %in% aiai)]
      vector <- unique(vector)
    }
    else {next}
  }
  #for each consequence, we write down its numerical impact
  for (hello in 1:length(vector)) {
    vector[hello] <- conseqnum[conseqnum$Consequence == vector[hello],"Impact"]
  }
  #now we save every row with this snp, but we only need one snp, since all of the consequences will be 
    #written down anyway and we have another column for the maximum consequence
  wewe <- vepresults[vepresults$`#Uploaded_variation`==rsid,]
  wewe <- wewe[1,]
  wewe[,15] <- paste(vector, collapse = ",")
  wewe[,16] <- vector[which.min(vector)]
  finalconseqnumbertable <- rbind(finalconseqnumbertable,wewe)
}

In [ ]:
finalconseqnumbertable <- fread("/home/aleixcanalda/Desktop/AleixCanalda/VEP/.txt", header=T)

## RegulomeDB

Next, our aim was to find out what effect each SNP had on regulatory region, if they found themselves in one, since they might not seem to have a big effect according to VEP, but regulatory regions are very important and must have a closer look. The scores for such regions range from 1a to 7, 1a being a SNP which presents a great effect on a regulatory region, and 7 not presenting a significant or no effect at all.

In [ ]:
#packages and libraries
install.packages("data.table")
install.packages("haploR", dependencies = TRUE)
library(data.table)
library(haploR)

In [ ]:
#we must read all of the rsIDs that we've used until now
dbsnp <- readLines("/home/aleixcanalda/Desktop/AleixCanalda/rsids.txt")
#now we bring our latest table so that we can add the new regulome information
finalconseqnumbertable <- fread("/home/aleixcanalda/Desktop/AleixCanalda/VEP/finalconseqtable.txt",header = T,
                                sep = "\t")
#we must create a regulome column where we'll write all of the scores for each SNP, the lowest number being a
#SNP highly related to any aspect of genome regulation
finalconseqnumbertable$Regulomescore <- NA
#we must make the whole column a character because we have both numbers and letters in our scores
finalconseqnumbertable[, `Regulomescore`:=as.character(`Regulomescore`)]

#we then create a loop where we go through each SNP to get their score
for (snp in dbsnp) {
  print(paste("-->", snp, sep=" "))
  #using this command we can access the database through R
  x <- queryRegulome(snp)
  #if we can't find a score for that SNP, it is probably unrelated to regulation or hasn't been looked at yet,
    #so we move on to the next one
  if (is.null(x$res.table$score)) {next}
  #but if we do indeed find a information for the SNP we must retain it in the rightful column
  else{
    finalconseqnumbertable[finalconseqnumbertable$`#Uploaded_variation`==snp, "Regulomescore"] <- 
      as.character(x$res.table$score)
  }
}
write.table(finalconseqnumbertable, file="/home/aleixcanalda/Desktop/AleixCanalda/VEP/finalconseqregulometable.txt",
            append = FALSE, quote = F, sep = "\t",
            col.names = TRUE, row.names = F)


Since we're interested in ranking the SNPs, like the consequences, we'll rank the regulome scores from 1 to 15, instead of 1a, 1b, 1c, ... 

In [ ]:
finalconseqnumbertable <- fread("/home/aleixcanalda/Desktop/AleixCanalda/VEP/finalconseqregulometable.txt",header = T,
                          sep = "\t")
finalconseqnumbertable$Regulomescore <- NA
#we must make the whole column a character because we have both numbers and letters in our scores
finalconseqnumbertable[, `Regulomescore`:=as.integer(`Regulomescore`)]

finalconseqnumbertable[,17] <- finalconseqtable[,15]

#to make our table even better we should separate the location column into chromosome and position since it's 
#generally more useful for handling data
finalconseqnumbertable$Chromosome <- NA
finalconseqnumbertable[, `Chromosome`:=as.character(`Chromosome`)]

#to do so we create a loop that goes through every row and separates the location column
for (chr in 1:nrow(finalconseqnumbertable)) {
  vec <- unlist(strsplit(as.character(finalconseqnumbertable[chr,2]),":"))
  finalconseqnumbertable[chr,18] <- vec[1]
  finalconseqnumbertable[chr,2] <- vec[2]
}
finalconseqnumbertable <- finalconseqnumbertable[,c(1,18,2:17)]

#now we must transform all of the regulome scores into numerical values in order to create a matrix for the heatmap
allscores <- c("1a","1b","1c","1d","1e","1f","2a","2b","2c","3a","3b","4","5","6","7")

#as we did before with the consequences we create a data.frame with the scores
conseqregscores <- data.frame(Scores=allscores,Importance=c(1:15))
#we then create a loop going through each snp and each score
for (regreg in 1:nrow(finalconseqnumbertable)) {
  for (eachscore in allscores) {
    #if the score is different from NA, then go through everything
    if (!is.na(finalconseqnumbertable[regreg,18])) {
      #if we have found our score, change into its numerical equivalent
      if (finalconseqnumbertable[regreg,18] == eachscore) {
        finalconseqnumbertable[regreg,18] <- conseqregscores[conseqregscores$Scores==eachscore,"Importance"]
      }
      #otherwise, go to the next score
      else {next}
      break
    }
    #if there is indeed an NA, go to the next snp
    else {next}
  }
}

write.table(finalconseqnumbertable, file="/home/aleixcanalda/Desktop/AleixCanalda/VEP/finalconseqnumbertable.txt",
            append = FALSE, quote = F, sep = "\t",
            col.names = TRUE, row.names = F)

## Heatmap

Now that we have all of the information that we need regarding VEP and Regulome scores, we can create a graph to represent the information and see more clearly which SNPs seem to have a bigger effect.

In [ ]:
#Packages and libraries
library(data.table)

library(gplots)

library(heatmap.plus)

library(RColorBrewer)

library(dplyr)

library(gridExtra)

library(ComplexHeatmap)

library(circlize)

In [ ]:
#we need the following tables in order to produce the graph
finalconseqnumbertable <- fread("/home/aleixcanalda/Desktop/AleixCanalda/VEP/finalconseqnumbertable.txt",header = T,
                                sep = "\t")
bigtable <- fread("/home/aleixcanalda/Desktop/AleixCanalda/goodstats.txt",header = T, sep = "\t") 
populations <- readLines("/home/aleixcanalda/Desktop/AleixCanalda/Populations.txt")

We're going to obtain a heatmap for every region with all of the SNPs inside that region.

In [ ]:
#in order to create all of the graphs at once we must do so for every chromosome
for (c in 1:22) {
  chromosome <- paste0("chr",c)
  print(paste("-->", chromosome, sep=" "))
  #the heat maps are created by regions according to PopHumanScan, therefore from the previously rearranged final 
    #consequence table we'll grab the snps in each round which are of interest to us, for each region for each 
    #chromosome
  #We create a subset of all the regions in PHS of that chromosome
  regions.per.chr <- bigtable %>% filter(chr==chromosome)
  for (region in 1:nrow(regions.per.chr)) {
    print(paste("--> region ", region, sep=" "))
    #we create a dataframe with which we'll create the desired heat map
    inregion <- data.frame()
    for (wowo in 1:nrow(finalconseqnumbertable)) {
      #if the snp we're looking at now is inside the region and in the chr we're looking at now, we'll keep it
      if (finalconseqnumbertable[wowo,3]>regions.per.chr[region,4] & finalconseqnumbertable[wowo,3]<
          regions.per.chr[region,5]) {
        lala <- finalconseqnumbertable[wowo,]
        inregion <- rbind(inregion,lala)
       
      }
    }
    #to create the matrix, we're solely interested in representing the VEP and RegulomeDB results, but we'll 
      #also need the location to compare each snp with its population and its rsID to classify each SNP
    if (length(inregion)==0) {next}
    inregion <- inregion[,c(1,3,17:18)]
    

    #we'll go through every population to see if it presents the SNP
    for (popu in 1:length(populations)) {
      print(paste("-->", populations[popu], sep=" "))
      #first we load the whole list of snps for that population
      prova.chr.pop <- fread(paste0('/home/aleixcanalda/Desktop/AleixCanalda/ihs.output/', chromosome,'/', chromosome,
                                    populations[popu],".ihs.out.100bins.norm"), header=T, sep="\t")
      #we create a dataframe where we'll save each snp whether its in the population or not(value of 0)
      popihs <- data.frame()
      for (snp in 1:nrow(inregion)) {
        print(paste("--> snp ", snp, sep=" "))
        
        #then we get the exact position of our current snp
        whatwhat <- unlist(inregion[snp,2])
        #if our snp is found in the document with all the snps looking at the location, then we'll save the iHS 
          #value of that snp
        if (is.element((inregion[snp,"Location"]),prova.chr.pop$physicalPos)){
         popihs <- rbind(popihs,unlist(prova.chr.pop[prova.chr.pop$physicalPos == whatwhat,"standarizediHS"]))
        }
        #if the snp isnt in the file, simply write a 0 (it'll appear white on the heatmap)
        else {popihs <- rbind(popihs,NA)}
      }
      #we'll name this dataframe with the population we're looking at now and we'll add it to the main heatmap 
        #dataframe
      colnames(popihs) <- populations[popu]
      inregion <- cbind(inregion,popihs)
    }
    #we want the rownames to be the rsIDs so we can identify each SNP later on
    rnames <-  inregion[,"#Uploaded_variation"]
    #once we've saved the rsIDs, we dont need them or their position anymore, so we only keep the rest
    inregion <- inregion[,3:26]
    rownames(inregion) <- unlist(rnames)
    #heatmap only works mith a matrix so we must transform our data into a matrix
    inregion <- data.matrix(inregion)
    
    eff <- inregion[,1:2]
    ihsmap <- inregion[,3:24]
    
    #in order to both create and save the heatmap for each region, we must firstly establish where we'll save it
    setwd(paste0("/home/aleixcanalda/Desktop/AleixCanalda/Heatmaps/",chromosome,"/"))
    #then we decide which type of document we'll be saving, an image in our case
    png(file=(paste0(chromosome,"region",region,"snppopeffect.png")),width=1000, height=1000)
    
    #we can then create our heatmap using the grid tools which will allow us to join in one same picture two 
      #heatmaps(the effects of the snps and the ihs of the populations)
    #inside viewport we decide where we want our heatmaps and determine their width and height
    #inside Heatmap, we actually create the graph and determine how we want the heatmap to be and what to 
      #include/exclude
    col_fun = colorRamp2(c(1, 10, 19), c("red", "orange", "white"))
    grid.newpage()
    pushViewport(viewport(x = 0.005, y=1, width = 1.1, height= 0.25, just = c("left", "top")))
    ht_list = Heatmap(t(eff),cluster_rows = F,cluster_columns = F, col=col_fun, name="SNP effect",row_names_gp =
                      gpar(fontsize = 10))
    draw(ht_list, newpage = FALSE)
    popViewport()
    
    pushViewport(viewport(x = 0.005, y=0.6, width = 1.012, height = 0.5, just = c("left", "top")))
    ht_list = Heatmap(t(ihsmap), col=colorRamp2(c(-4, 0, 4), c("blue", "white", "red")),cluster_rows = 
                      F,cluster_columns = F, name = "Pop iHS")
    draw(ht_list, newpage = FALSE)
    popViewport()
    
    dev.off()
    
    #here I present an example of how to create an interactive map , which may or may not be useful at some point
    #library("d3heatmap")
    #d3heatmap(scale(t(ihsmap)),  main="iHS in Populations", Rowv=NA,Colv=NA, cexRow=1, cexCol=1,
    #          key = TRUE, keysize = 1,margins = c(25,15), trace="none", lhei = c(5,20), lwid = c(2,10), 
      #col=bluered(20),
    #          k_row = 4, # Number of groups in rows
    #          k_col = 2) # Number of groups in columns
    
  }
}

The result we would obtain would be something similar to:(heatmap example)

## Order

Once all the above has been done (which may take a while), in order to see which SNP has both a high adaptive significance (high iHS) and a high consequence/regulomescore, which means that that SNP has an important role, we should create new tables with which we can correlate these factors.

In [ ]:
finalconseqnumbertable$iHS <- NA
transform(finalconseqnumbertable,'iHS' = as.numeric('iHS'))
finalconseqnumbertable$iHSPop <- NA
finalconseqnumbertable[, `iHSPop`:=as.double(`iHSPop`)]
finalconseqnumbertable$iHSxRegulome <- NA
transform(finalconseqnumbertable,'iHSxRegulome' = as.numeric('iHSxRegulome'))
finalconseqnumbertable$iHSxConsequence <- NA
transform(finalconseqnumbertable,'iHSxConsequence' = as.numeric('iHSxConsequence'))

Now we will go again chromosome for chromosome adding these values.

In [ ]:
#firstly we must create the iHS and iHS with its population simultaneously
for (c in 1:22) {
  chromosome <- paste0("chr",c)
  print(paste("-->", chromosome, sep=" "))
  finalconseqchr <- finalconseqnumbertable %>% filter(Chromosome==c)
    #we'll go through every population 
    for (popu in 1:length(populations)) {
      print(paste("-->", populations[popu], sep=" "))
      #we'll get the file where we can find the iHS values
      prova.chr.pop <- fread(paste0('/home/aleixcanalda/Desktop/AleixCanalda/ihs.output/', chromosome,'/',
                                    chromosome, populations[popu],".ihs.out.100bins.norm"), header=T, sep="\t")
      
      #then we'll go snp for snp
      for (snp in 1:nrow(finalconseqchr)) {
        print(paste("--> snp ", snp, sep=" "))
        #we get the exact position of our current snp
        whatwhat <- unlist(finalconseqchr[snp,3])
        #if our snp is found in the document with all the snps looking at the location, we continue, if not,
          #we move on to the next snp
        if (is.element((finalconseqchr[snp,"Location"]),prova.chr.pop$physicalPos)){
          #if the snp written on the table so far is smaller than the one we're looking at now or IS NA, we
            #save the new iHS value with its population
          if ((abs(finalconseqnumbertable[finalconseqnumbertable$Chromosome==c & 
                                          finalconseqnumbertable$Location==whatwhat,"iHS"]) < 
               abs(unlist(prova.chr.pop[prova.chr.pop$physicalPos == whatwhat,"standarizediHS"])))
               | is.na(finalconseqnumbertable[finalconseqnumbertable$Chromosome==c & 
                                              finalconseqnumbertable$Location==whatwhat,"iHS"])) {
          finalconseqnumbertable[finalconseqnumbertable$Chromosome==c & finalconseqnumbertable$Location==whatwhat,19]
              <- unlist(prova.chr.pop[prova.chr.pop$physicalPos == whatwhat,"standarizediHS"])
          finalconseqnumbertable[finalconseqnumbertable$Chromosome==c & finalconseqnumbertable$Location==whatwhat,20] 
              <- populations[popu]
          }
          else {next}
        }
        
        else {next}
      }
       
    }
}
#after we have the iHS and iHSPop values, we fill the other two columns with the quocient mentioned beforehand
for(yalla in 1:nrow(finalconseqnumbertable)) {
  finalconseqnumbertable[yalla,21] <- (finalconseqnumbertable[yalla,18] / finalconseqnumbertable[yalla,19])
  finalconseqnumbertable[yalla,22] <- (finalconseqnumbertable[yalla,17] / finalconseqnumbertable[yalla,19])
}

After having created the new ordered table, we want to merge the table with the regions with the table with the snps so that we can associate them and order the regions using the snps to see which regions seem to be more important

In [ ]:
finalbigtable <- data.frame()
for (c in 1:22) {
  chromosome <- paste0("chr",c)
  print(paste("-->", chromosome, sep=" "))
  # We create a subset of all the regions in PHS of that chromosome
  regions.per.chr <- bigtable %>% filter(chr==chromosome)
  snp.per.chr <- finalconseqnumbertable %>% filter(Chromosome==c)
  for (region in 1:nrow(regions.per.chr)) {
    print(paste("--> region ", region, sep=" "))
    for (snp in 1:nrow(snp.per.chr)) {
      yey <- data.frame()
      print(paste("-->", snp, sep=" "))
      if (snp.per.chr[snp,3]>regions.per.chr[region,5] & snp.per.chr[snp,3]<regions.per.chr[region,6]) {
        yey <- rbind(yey,bigtable[605,])
        yey <- cbind(yey,finalconseqnumbertable[26023,])
        finalbigtable <- rbind(finalbigtable,yey)
      }
      else{next}
    }
  }
}